#Set-up

In [1]:
from google.colab import drive
drive.mount('/content/drive')

!pip install transformers

Mounted at /content/drive


In [2]:
# set seeds
import random
import numpy as np
import torch

def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)

  torch.backends.cudnn.deterministic = True #- Forces cuDNN to use deterministic algorithms
  torch.backends.cudnn.benchmark = False #

set_seed(42)

In [12]:
# Add the specific directory to the Python path
import sys
sys.path.append('/content/drive/MyDrive/generally_useful_scripts')

# Import all functions from utils_addgene
from utils_addgene import *
import utils_addgene
import importlib
importlib.reload(utils_addgene)


## load custom functions from utils.py

import sys
sys.path.append('//content/drive/MyDrive/SAEs_for_Genomics')

import importlib
import utils
importlib.reload(utils)

<module 'utils' from '//content/drive/MyDrive/SAEs_for_Genomics/utils.py'>

# Load NT model

In [4]:
"loading smallest nucleotide transformer (50m params)"
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoConfig
import torch

num_params = 50 ## default 50

# Import the tokenizer and the model
tokenizer_nt = AutoTokenizer.from_pretrained(f"InstaDeepAI/nucleotide-transformer-v2-{num_params}m-multi-species", trust_remote_code=True)
model_nt = AutoModelForMaskedLM.from_pretrained(f"InstaDeepAI/nucleotide-transformer-v2-{num_params}m-multi-species", trust_remote_code=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/129 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/28.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/101 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

esm_config.py:   0%|          | 0.00/14.9k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/nucleotide-transformer-v2-50m-multi-species:
- esm_config.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_esm.py:   0%|          | 0.00/58.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/InstaDeepAI/nucleotide-transformer-v2-50m-multi-species:
- modeling_esm.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/224M [00:00<?, ?B/s]

In [5]:
# Optionally we can load weights from extra pretraining
# lets load addgene-pretrained weights
model_nt_extra_pre = model_nt
weights_path = '/content/drive/MyDrive/SAEs_for_Genomics/nt_weights_extra_pretraining/NT50Mm_addgenepretrained_epoch_4.pt'
model_nt_extra_pre.load_state_dict(torch.load(weights_path)['model_state_dict'])

<All keys matched successfully>

# Set-up & Load SAE

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

cfg = {
    "seed": 49,
    "batch_size": 4096,
    "buffer_mult": 384,
    "lr": 5e-5,
    "num_tokens": tokenizer_nt.vocab_size,
    "d_model": model_nt.config.hidden_size,
    "l1_coeff": 1e-1,
    "l0_coeff": 1,
    "beta1": 0.9,
    "beta2": 0.999,
    "dict_mult": 8, # hidden_d = d_model * dict_mult
    "seq_len": 512,
    "d_mlp": model_nt.config.hidden_size,
    "enc_dtype":"fp32",
    "remove_rare_dir": False,
    "total_training_steps": 10000,
    "lr_warm_up_steps": 1000,
    "device": "cuda",
    "tempterature":0.05,
    "activation_treshold":0.3,
    "enc_dtype": torch.float32

}

sae_model = utils.AutoEncoder(cfg)

In [ ]:
#count params of sae_model
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'Number of parameters in sae_model: {count_parameters(sae_model)}')

Number of parameters in sae_model: 4198912


# Load and preprocess addgene dataset

In [9]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np


# Constants
TEST_DATA_PATH = '/content/drive/MyDrive/NOO_paper/datasets/worldwide/blast_geac_ext_169k_val_random.csv'
TRAIN_DATA_PATH = '/content/drive/MyDrive/NOO_paper/datasets/worldwide/blast_geac_ext_169k_train_random.csv'
INFREQUENT_THRESHOLD = 10

# Preprocess data: each data point has shape (DNA_seq: str, nation_idx: int)
df_train, df_val = utils_addgene.preprocess_data(TRAIN_DATA_PATH, TEST_DATA_PATH)

# Display the split data
print("Train Data Shape:", df_train.shape)
print("Validation Data Shape:", df_val.shape)

# Create torch datasets in which sequences are tokenized
# Each data point has shape (input_ids, attention_masks, labels)
val_dataset = GenomicDataset(df_val, tokenizer_nt=tokenizer_nt)
train_dataset = GenomicDataset(df_train, tokenizer_nt=tokenizer_nt)

BS = 64*6

val_loader_dna = DataLoader(val_dataset, batch_size=BS, shuffle=False, pin_memory=True, num_workers=2)
train_loader_dna = DataLoader(train_dataset, batch_size=BS, shuffle=True, pin_memory=True, num_workers=2)

y_test shape: (15551,)
test_data shape: (15551, 4)
Train Data Shape: (93306, 2)
Validation Data Shape: (15551, 2)


# Test-run SAE



In [8]:
## test run SAE on batch of data
model_nt_extra_pre.eval().cuda()
for batch in train_loader_dna:

    input_ids = batch['input_ids'].cuda()
    attention_mask = batch['attention_mask'].cuda()
    label = batch['label'].cuda()

    # forward pass with model_nt to get acts
    with torch.no_grad():
        mlp_act = utils.get_layer_activations(model_nt_extra_pre, input_ids, attention_mask, layer_N=11)
        acts = mlp_act[0].reshape(-1, model_nt_extra_pre.config.hidden_size)

    with torch.cuda.amp.autocast():
        loss, x_reconstruct, acts_sparse, l2_loss, nmse, l1_loss, true_l0 = sae_model(acts)


    print(f'loss: {loss}')
    print(f'l2_loss: {l2_loss}')
    print(f'nmse: {nmse}')
    print(f'l1_loss: {l1_loss}')
    break


<ipython-input-8-44de7ea9adb1>:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


loss: 1955.95849609375
l2_loss: 984.78076171875
nmse: 1.386866807937622
l1_loss: 609.4150390625


# Extracting Activations

In [10]:
import torch
from tqdm import tqdm
import h5py
import numpy as np
from torch.cuda.amp import autocast
import utils

def extract_activations(model_nt, dataloader, layer_N=11, save_path='activations.h5'):
    model_nt.eval()
    all_activations = []

    with h5py.File(save_path, 'w') as f:
        # Create dataset with unknown size initially
        dset = None
        current_idx = 0

        with torch.no_grad():
            total_batches = len(dataloader)
            progress_bar = tqdm(total=total_batches, desc="Extracting activations")
            for batch in dataloader:
                input_ids = batch['input_ids'].cuda()
                attention_mask = batch['attention_mask'].cuda()

                with autocast():
                    mlp_act = utils.get_layer_activations(model_nt,
                                                        input_ids,
                                                        attention_mask,
                                                        layer_N=layer_N)

                mlp_act = mlp_act[0].reshape(-1, model_nt.config.hidden_size)
                mlp_act = mlp_act.cpu().numpy()
                print(mlp_act.shape)

                # Initialize dataset with correct shape if first batch
                if dset is None:
                    total_size = len(dataloader) * mlp_act.shape[0]
                    dset = f.create_dataset('activations',
                                          shape=(total_size, mlp_act.shape[1]),
                                          dtype='float32',
                                          chunks=True)

                # Save batch
                dset[current_idx:current_idx + mlp_act.shape[0]] = mlp_act
                current_idx += mlp_act.shape[0]

                # Clear memory
                del mlp_act, input_ids, attention_mask
                torch.cuda.empty_cache()

                progress_bar.update(1)


# Run extraction
model_nt.eval()
extract_activations(model_nt.cuda(),
                    train_loader_dna,
                    save_path='/content/drive/MyDrive/SAEs_for_Genomics/plasmidpretrained_NT50_L12_mlp.activations.h5')

Extracting activations:   0%|          | 0/243 [00:00<?, ?it/s]<ipython-input-10-f8e055596b81>:24: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


(196608, 512)


Extracting activations:   0%|          | 1/243 [00:16<1:04:55, 16.10s/it]

(196608, 512)


Extracting activations:   1%|          | 2/243 [00:31<1:03:04, 15.70s/it]

...

Extracting activations:  99%|█████████▉| 241/243 [59:34<00:29, 14.59s/it]

(196608, 512)


Extracting activations: 100%|█████████▉| 242/243 [59:49<00:14, 14.75s/it]

(193536, 512)


Extracting activations: 100%|██████████| 243/243 [1:00:03<00:00, 14.83s/it]


# Training SAE

In [13]:
importlib.reload(utils)

# Enable anomaly detection
torch.autograd.set_detect_anomaly(True)

# Loads chunks of the dataset at a time
file_path = '/content/drive/MyDrive/SAEs_for_Genomics/plasmidpretrained_NT50_L12_mlp.activations.h5'
start_chunk = 0
dataset = utils.ChunkedActivationsDataset(file_path)

# Train sae_model on dataset chunks
sae_model = utils.train_sae(
        sae_model=sae_model,
        dataset=dataset,
        batch_size=2048*4,
        num_epochs=20,
        learning_rate=5e-5,
        device='cuda',
        start_chunk = start_chunk,
        wandb_log= False,
        save_dir='/content/drive/MyDrive/SAEs_for_Genomics/sae_weights/SAE_NT50_plasmidpre_L12_mlpout_40mtokens_190325.pt'
)



Epoch 1/20:   0%|          | 0/5832 [00:00<?, ?it/s]

IndexError: index 13620 is out of bounds for dimension 0 with size 0

In [15]:
# save sae weights and HPs
import os
import torch

model_dict = {
    'model_state_dict': sae_model.state_dict(),
    'hyperparameters': cfg
}

#save
torch.save(model_dict, '/content/drive/MyDrive/SAEs_for_Genomics/sae_weights/extra_addgene_pretraining/SAE_NT50_plasmidpre_L12_mlpout_40mtokens_190325.pt')


# HP Tuning

In [ ]:
### wandb init
!pip install wandb
import wandb

# Set your API key here
API_KEY = "9d4aba4e2ab3fcfc7278c20827aacff0f7fca089"

def initialize_wandb(run_name, hyperparameter, reinit, project = "DeepGEA"):
    wandb.login(key=API_KEY)  # Ensure API key is stored securely
    wandb.init(project=project, entity="amaiwald", name=run_name, mode = 'online', config=hyperparameter, reinit=reinit)

run_name = 'SAE_random.NT50_L11.mlp_48mtokens' ## choose
#initialize_wandb(run_name = run_name, hyperparameter = cfg)


In [ ]:
# randomly search grid of HPs and train each for 20 ep
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import ParameterSampler
import numpy as np
from datetime import datetime
import json
import traceback

param_distributions = {
    'lr': np.log10(np.array([5e-6, 1e-3])),  # log-uniform between 5e-6 and 1e-3
    'l0_weight': np.log10(np.array([0.001, 2.0])),  # log-uniform between 0.001 and 2.0
    'threshold': np.array([0.03, 2.4]),  # uniform between 0.03 and 2.4
    'dict_mult': [8, 16, 32]  # keep discrete values
}

def sample_params(n_samples=10, random_seed=42):
    samples = list(ParameterSampler(param_distributions, n_samples, random_state=random_seed))
    for sample in samples:
        sample['lr'] = 10 ** sample['lr']
        sample['l0_weight'] = 10 ** sample['l0_weight']
    return samples


# Create full training dataset
dataset = ChunkedActivationsDataset(
    '/content/drive/MyDrive/SAEs_for_Genomics/plasmidpretrained_NT50_L11_mlp.activations.h5',
    batch_size=2048*4,
    chunks_in_memory=20  # Adjust based on your GPU memory
)

# Test the sampling
samples = sample_params(n_samples=20, random_seed=42)
print(samples)

# Constants
N_EPOCHS = 10


# In your main script
for i, sample in enumerate(samples):
    # Create unique directory for this trial
    trial_dir = os.path.join('/content/drive/MyDrive/SAEs_for_Genomics', f'NT.plasmidpretr_SAE_trial_{i}')
    os.makedirs(trial_dir, exist_ok=True)

    # Save trial config
    trial_config = {
        'trial_id': i,
        'hyperparameters': sample,
        'timestamp': datetime.now().isoformat()
    }
    with open(os.path.join(trial_dir, 'config.json'), 'w') as f:
        json.dump(trial_config, f, indent=2)

    # Set HPs for this trial
    cfg['lr'] = sample['lr']
    cfg['l0_weight'] = sample['l0_weight']
    cfg['threshold'] = sample['threshold']
    cfg['dict_mult'] = sample['dict_mult']

    print(f"\nStarting trial {i+1}/20")
    print("HPs:", cfg)
    print(50*"-")

    # Initialize model and W&B
    sae_model = AutoEncoder(cfg)
    run_name = f'SAE_NT50.plasmidpretr_L11.mlp_48mtokens_trial_{i}'
    initialize_wandb(run_name=run_name, hyperparameter=cfg, reinit=True)

    try:
        results = train_sae(
            sae_model=sae_model,
            dataset=dataset,
            batch_size=2048*4,
            num_epochs=N_EPOCHS,
            learning_rate=cfg['lr'],
            device='cuda',
            start_chunk=start_chunk,
            wandb_log=True,
            save_dir=trial_dir  # Save in trial-specific directory
        )

        sae_model = results['model']
        best_loss = results['best_loss']
        best_epoch = results['best_epoch']
        final_metrics = results['final_metrics']

        # Save final model state with hyperparameters
        final_save_path = os.path.join(trial_dir, 'final_model.pt')
        torch.save({
            'trial_id': i,
            'hyperparameters': sample,
            'model_state_dict': sae_model.state_dict(),
            'final_metrics': {
                'best_loss': best_loss,  # You'll need to modify train_sae to return this
                'final_epoch': N_EPOCHS
            }
        }, final_save_path)

    except Exception as e:
        print(f"Trial {i+1} failed with error: {str(e)}")
        with open(os.path.join(trial_dir, 'error_log.txt'), 'w') as f:
            f.write(f"Error: {str(e)}\n")
            f.write(traceback.format_exc())

    finally:
        wandb.finish()
        print(f"Cleaned up trial {i+1}")



wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


[{'threshold': 0.03, 'lr': 4.9999999999999996e-06, 'l0_weight': 0.001, 'dict_mult': 16}, {'threshold': 0.03, 'lr': 4.9999999999999996e-06, 'l0_weight': 0.001, 'dict_mult': 32}, {'threshold': 0.03, 'lr': 4.9999999999999996e-06, 'l0_weight': 0.001, 'dict_mult': 8}, {'threshold': 0.03, 'lr': 0.001, 'l0_weight': 0.001, 'dict_mult': 32}, {'threshold': 2.4, 'lr': 0.001, 'l0_weight': 0.001, 'dict_mult': 16}, {'threshold': 2.4, 'lr': 4.9999999999999996e-06, 'l0_weight': 0.001, 'dict_mult': 16}, {'threshold': 2.4, 'lr': 4.9999999999999996e-06, 'l0_weight': 2.0, 'dict_mult': 16}, {'threshold': 2.4, 'lr': 4.9999999999999996e-06, 'l0_weight': 0.001, 'dict_mult': 8}, {'threshold': 2.4, 'lr': 4.9999999999999996e-06, 'l0_weight': 2.0, 'dict_mult': 32}, {'threshold': 2.4, 'lr': 4.9999999999999996e-06, 'l0_weight': 2.0, 'dict_mult': 8}, {'threshold': 0.03, 'lr': 0.001, 'l0_weight': 0.001, 'dict_mult': 8}, {'threshold': 0.03, 'lr': 4.9999999999999996e-06, 'l0_weight': 2.0, 'dict_mult': 16}, {'threshold'

Epoch 1, Chunk 1/293:   5%|▌         | 1/20 [00:00<00:11,  1.62it/s, loss=2458.4248, l2_loss=901.9199, l1_loss=774.8301, l0_loss=1556.5049, nmse=1.3272]

Input stats:
Input range: [-215.5000, 69.0000]
Input mean/std: 0.0184 / 11.7914

Activations after encoding:
Acts range: [0.0000, 1.8428]
Acts mean/std: 0.1378 / 0.2050

Sparse activations:
Sparse acts range: [0.0000, 1.8428]
Sparsity: 80.9697%

Reconstruction stats:
Recon range: [-4.0586, 4.3750]
Recon mean/std: -0.0100 / 0.8696
Input stats:
Input range: [-214.2500, 66.0625]
Input mean/std: 0.0165 / 11.8339

Activations after encoding:
Acts range: [0.0000, 1.8193]
Acts mean/std: 0.1377 / 0.2050

Sparse activations:
Sparse acts range: [0.0000, 1.8193]
Sparsity: 80.9997%

Reconstruction stats:
Recon range: [-3.9180, 4.2734]
Recon mean/std: -0.0099 / 0.8691


Epoch 1, Chunk 1/293:  15%|█▌        | 3/20 [00:00<00:03,  4.37it/s, loss=2459.9651, l2_loss=902.2513, l1_loss=775.2341, l0_loss=1557.7137, nmse=1.3275]

...

Recon range: [-3.6445, 4.1367]
Recon mean/std: -0.0066 / 0.8257
Input stats:
Input range: [-213.5000, 66.3750]
Input mean/std: 0.0174 / 11.8377

Activations after encoding:
Acts range: [0.0000, 1.8203]
Acts mean/std: 0.1367 / 0.2041

Sparse activations:
Sparse acts range: [0.0000, 1.8203]
Sparsity: 81.1526%

Reconstruction stats:
Recon range: [-3.6582, 4.0859]
Recon mean/std: -0.0060 / 0.8198
Input stats:
Input range: [-213.1250, 66.1250]
Input mean/std: 0.0169 / 11.8000

Activations after encoding:
Acts range: [0.0000, 1.8213]
Acts mean/std: 0.1367 / 0.2039

Sparse activations:
Sparse acts range: [0.0000, 1.8213]
Sparsity: 81.1317%

Reconstruction stats:
Recon range: [-3.8125, 4.0547]
Recon mean/std: -0.0059 / 0.8145


Epoch 1, Chunk 1/293:  95%|█████████▌| 19/20 [00:02<00:00, 11.73it/s, loss=2329.0815, l2_loss=787.2192, l1_loss=765.5928, l0_loss=1541.8623, nmse=1.2400]

Input stats:
Input range: [-212.5000, 66.7500]
Input mean/std: 0.0183 / 11.8541

Activations after encoding:
Acts range: [0.0000, 1.8379]
Acts mean/std: 0.1365 / 0.2039

Sparse activations:
Sparse acts range: [0.0000, 1.8379]
Sparsity: 81.1909%

Reconstruction stats:
Recon range: [-3.8086, 3.8848]
Recon mean/std: -0.0053 / 0.8081
Input stats:
Input range: [-212.8750, 66.3750]
Input mean/std: 0.0170 / 11.8189

Activations after encoding:
Acts range: [0.0000, 1.7988]
Acts mean/std: 0.1365 / 0.2037

Sparse activations:
Sparse acts range: [0.0000, 1.7988]
Sparsity: 81.1784%

Reconstruction stats:
Recon range: [-3.6504, 3.9355]
Recon mean/std: -0.0052 / 0.8032
Input stats:
Input range: [-211.8750, 67.2500]
Input mean/std: 0.0160 / 11.8548

Activations after encoding:
Acts range: [0.0000, 1.7949]
Acts mean/std: 0.1362 / 0.2036

Sparse activations:
Sparse acts range: [0.0000, 1.7949]
Sparsity: 81.2209%

Reconstruction stats:
Recon range: [-3.6074, 3.8867]
Recon mean/std: -0.0046 / 0.7974

Epoch 1, Chunk 1/293: 100%|██████████| 20/20 [00:02<00:00,  8.71it/s, loss=2315.5640, l2_loss=777.1824, l1_loss=763.9532, l0_loss=1538.3817, nmse=1.2320]


Epoch 1, Chunk 2/293:   0%|          | 0/20 [00:00<?, ?it/s]

### test HP run

In [ ]:
initialize_wandb(run_name=run_name, reinit = True, hyperparameter=cfg, project="test_genomics_sae")  # Note the test_ prefix


# Test version of your code
def test_hp_search(n_trials=2, n_epochs=2):
    # Minimal parameter space
    test_param_distributions = {
        'lr': np.log10(np.array([1e-4, 1e-3])),  # just two values
        'l0_weight': np.log10(np.array([0.1, 0.2])),
        'threshold': np.array([0.1, 0.2]),
        'dict_mult': [8]  # single value
    }
    BATCH_SIZE = 2048*4

    test_dataset = ChunkedActivationsDataset(
    '/content/drive/MyDrive/SAEs_for_Genomics/plasmidpretrained_NT50_L11_mlp.activations.h5',
    batch_size=BATCH_SIZE,  # Your actual batch size
    chunks_in_memory=4,  # Each chunk holds 4 batches worth of data
    max_chunks=10  # Limit total chunks for testing
    )

    samples = sample_params(n_samples=n_trials)

    for i, sample in enumerate(samples):
        print(f"Test trial {i+1}/{n_trials}")

        cfg['lr'] = sample['lr']
        cfg['l0_weight'] = sample['l0_weight']
        cfg['threshold'] = sample['threshold']
        cfg['dict_mult'] = sample['dict_mult']

        sae_model = AutoEncoder(cfg)

        # Note the TEST prefix in run name
        run_name = f'TEST_run{i}'
        initialize_wandb(
            run_name=run_name,
            hyperparameter=cfg,
            reinit=True,
            project="test_genomics_sae"  # separate test project
        )


        try:
            # Use minimal data and epochs
            sae_model = train_sae(
                sae_model=sae_model,
                dataset=test_dataset,  # just use 100 samples
                batch_size=BATCH_SIZE,  # smaller batch
                num_epochs=n_epochs,  # minimal epochs
                learning_rate=cfg['lr'],
                device='cuda',
                start_chunk=start_chunk,
                wandb_log=True,
                save_dir='/tmp/test_sae/'  # temporary directory
            )
        finally:
            wandb.finish()
            print(f"Finished and cleaned up trial {i+1}")

# Run the test
test_hp_search(n_trials=5, n_epochs=2)



In [ ]:
def cleanup_test_runs():
    api = wandb.Api()
    runs = api.runs("amaiwald/test_genomics_sae")
    for run in runs:
        if run.name.startswith('TEST_'):
            print(f"Deleting test run: {run.name}")
            run.delete()

# Run cleanup after testing
cleanup_test_runs()

Deleting test run: TEST_run0
Deleting test run: TEST_run0
Deleting test run: TEST_run0
Deleting test run: TEST_run0
Deleting test run: TEST_run0
Deleting test run: TEST_run0
Deleting test run: TEST_run0
Deleting test run: TEST_run1
Deleting test run: TEST_run2
Deleting test run: TEST_run3
Deleting test run: TEST_run4


In [ ]:
## load custom functions from utils.py

import sys
sys.path.append('//content/drive/MyDrive/SAEs_for_Genomics')

import importlib
import utils
importlib.reload(utils)

<module 'utils' from '//content/drive/MyDrive/SAEs_for_Genomics/utils.py'>

In [ ]:
import torch
from torch.utils.checkpoint import checkpoint
import gc
import torch.nn.functional as F
from torch.optim.lr_scheduler import LambdaLR

def get_linear_schedule_with_warmup(optimizer, num_warmup_steps, num_training_steps):
    def lr_lambda(current_step):
        if current_step < num_warmup_steps:
            return float(current_step) / float(max(1, num_warmup_steps))
        return max(0.0, float(num_training_steps - current_step) / float(max(1, num_training_steps - num_warmup_steps)))
    return LambdaLR(optimizer, lr_lambda)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
optimizer = torch.optim.AdamW(sae_model.parameters(), lr=cfg['lr'], betas=(cfg['beta1'], cfg['beta2']))
total_steps = cfg['total_training_steps']
warmup_steps = cfg['lr_warm_up_steps']
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps, num_training_steps=total_steps)

# Enable gradient checkpointing for the transformer model
model_nt.to(device).eval()
sae_model.train()

num_layer = 11
d_mlp = model_nt.config.hidden_size
n_epochs = 20
global_step = 0
train_epoch_loss = 0
train_epoch_l2_loss = 0
train_epoch_l1_loss = 0
train_epoch_l0_loss = 0
train_epoch_nmse = 0
all_latents = []

for epoche in range(n_epochs):
    for batch in tqdm(train_dataloader, desc="Inference"):
        with torch.no_grad():
            # Move everything to device
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            mlp_act = utils.get_layer_activations(
                model_nt.cuda(),
                input_ids,
                attention_mask
            )

            mlp_act = mlp_act[0].reshape(-1, d_mlp)

            # Move to CPU temporarily to save GPU memory
            mlp_act = mlp_act.cpu()

        # Move back to GPU for SAE processing
        mlp_act = mlp_act.cuda()

        # Train SAE based on this
        optimizer.zero_grad()
        loss, x_reconstruct, acts, l2_loss, nmse, l1_loss, l0_loss = sae_model(mlp_act)
        loss.backward()

        # Explicit gradient cleanup
        for param in sae_model.parameters():
            if param.grad is not None:
                param.grad.detach_()
                param.grad.zero_()

        sae_model.remove_parallel_component_of_grads()
        optimizer.step()
        scheduler.step()

        train_epoch_loss += loss.item()
        train_epoch_l2_loss += l2_loss.item()
        train_epoch_l1_loss += l1_loss.item()
        train_epoch_l0_loss += l0_loss.item()
        train_epoch_nmse += nmse

        # Explicit cleanup
        del loss, x_reconstruct, acts, l2_loss, nmse, l1_loss, l0_loss
        del mlp_act
        torch.cuda.empty_cache()
        gc.collect()

        global_step += 1
        if global_step >= total_steps:
            break

    # Calculate average training losses
    avg_train_loss = train_epoch_loss / len(train_dataloader)
    avg_train_l2_loss = train_epoch_l2_loss / len(train_dataloader)
    avg_train_l1_loss = train_epoch_l1_loss / len(train_dataloader)
    avg_train_l0_loss = train_epoch_l0_loss / len(train_dataloader)
    avg_train_nmse = train_epoch_nmse / len(train_dataloader)

    print(f'Training - Epoch {epoche}, Loss: {avg_train_loss:.4f}, L2 Loss: {avg_train_l2_loss:.4f}, '
          f'L1 Loss: {avg_train_l1_loss:.4f}, L0 Loss: {avg_train_l0_loss:.4f}, NMSE: {avg_train_nmse:.4f}, '
          f'LR: {scheduler.get_last_lr()[0]:.6f}')

    # Reset metrics for next epoch
    train_epoch_loss = 0
    train_epoch_l2_loss = 0
    train_epoch_l1_loss = 0
    train_epoch_l0_loss = 0
    train_epoch_nmse = 0
    torch.cuda.empty_cache()

Inference:   0%|          | 0/23327 [00:00<?, ?it/s]

Exception ignored in: <function _xla_gc_callback at 0x7dbe53a839c0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 


KeyboardInterrupt: 

# Load pretraining data of NT

In [ ]:
!pip install datasets
!pip install huggingface_hub
!pip install biopython

## Load HF dataset for Streaming

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from datasets import load_dataset
from typing import Optional
import numpy as np
from tqdm import tqdm



class StreamingGenomicDataset(Dataset):
    def __init__(
        self,
        split: str = "train",
        tokenizer = None,
        seq_length: int = 512,
        max_samples: Optional[int] = None,
        cache_mode: bool = True
    ):
        """
        Streaming dataset for genomic sequences.

        Args:
            split: Dataset split ('train', 'validation', 'test')
            tokenizer: Tokenizer for DNA sequences
            seq_length: Maximum sequence length
            max_samples: Maximum number of samples to load (None for all)
            cache_mode: If True, caches all sequences in memory
        """
        self.seq_length = seq_length
        self.tokenizer = tokenizer

        # In the __init__ method:
        dataset = load_dataset("metagene-ai/HumanVirusInfecting", "class-1", streaming=True)
        # Get the specific split
        dataset = dataset[split]  # This was missing in your original code

        if cache_mode:
            # Cache all sequences in memory
            self.sequences = []
            pbar = tqdm(dataset, total=max_samples, desc=f"Loading {split} data")

            for i, item in enumerate(pbar):
                if max_samples and i >= max_samples:
                    break
                if not isinstance(item, dict):
                    print(f"Warning: item at index {i} is not a dictionary: {item}")
                    continue
                self.sequences.append(item)
        else:
            # Store iterator for streaming mode
            self.sequences = dataset
            self.max_samples = max_samples

    def __len__(self):
        if isinstance(self.sequences, list):
            return len(self.sequences)
        return self.max_samples if self.max_samples else int(1e9)  # Large number for streaming

    # First, let's modify the __getitem__ method to debug the data structure:

    def __getitem__(self, idx):
        if isinstance(self.sequences, list):
            # Cached mode
            item = self.sequences[idx]
        else:
            # Streaming mode
            item = next(iter(self.sequences))


        # Let's safely access the sequence
        if isinstance(item, dict):
            sequence = item.get('sequence')
        else:
            raise TypeError(f"Expected item to be dict, got {type(item)} instead")

        if sequence is None:
            raise ValueError("No 'sequence' field found in item")

        # Rest of your code remains the same
        inputs = self.tokenizer(
            sequence,
            max_length=self.seq_length,
            padding='max_length',
            truncation=True,
            return_tensors="pt"
        )

        return {
            'input_ids': inputs['input_ids'].squeeze(),
            'attention_mask': inputs['attention_mask'].squeeze(),
            'sequence_info': {
                'description': item.get('description', ''),
                'start_pos': item.get('start_pos', 0),
                'end_pos': item.get('end_pos', 0)
            }
        }

def create_genomic_dataloaders(
    tokenizer,
    batch_size: int = 32,
    seq_length: int = 512,
    max_samples: Optional[int] = None,
    num_workers: int = 2,
    cache_mode: bool = True
):
    """
    Create training and validation DataLoaders for genomic data.

    Args:
        tokenizer: DNA sequence tokenizer
        batch_size: Batch size for DataLoader
        seq_length: Maximum sequence length
        max_samples: Maximum samples per split (None for all)
        num_workers: Number of DataLoader workers
        cache_mode: If True, caches all sequences in memory
    """
    # Create datasets
    train_dataset = StreamingGenomicDataset(
        split="train",
        tokenizer=tokenizer,
        seq_length=seq_length,
        max_samples=max_samples,
        cache_mode=cache_mode
    )


    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=cache_mode,  # Can only shuffle if data is cached
        num_workers=num_workers,
        pin_memory=True
    )


    return train_loader

# Example usage
# Create dataloaders with small sample size for testing
train_loader = create_genomic_dataloaders(
    tokenizer=tokenizer,
    batch_size=16*3,
    seq_length=50,
    max_samples=19600,  # Small sample size for testing
    cache_mode=True
)

# Print dataset sizes
print(f"Training batches: {len(train_loader)}")

# Example of iterating through one batch
for batch in train_loader:
    print("\nBatch shapes:")
    print(f"Input ids: {batch['input_ids'].shape}")
    print(f"Attention mask: {batch['attention_mask'].shape}")
    break



Loading train data: 100%|██████████| 19600/19600 [00:01<00:00, 10101.86it/s]

Training batches: 409



Batch shapes:
Input ids: torch.Size([48, 50])
Attention mask: torch.Size([48, 50])


In [ ]:
from tqdm import tqdm  # Import tqdm

## test how long it takes for the model to perform forward passes on all of the train batches
model.eval()

for batch in tqdm(train_loader):  # Wrap train_loader with tqdm
    input_ids = batch['input_ids'].to(model.device)
    attention_mask = batch['attention_mask'].to(model.device)

    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

100%|██████████| 77/77 [01:22<00:00,  1.08s/it]
